# Holidays, where are they?!🎄

The interplay between holidays and weekends adds a layer of complexity to our yearly calendars. While the prospect of long weekends is often welcomed, the shifting sands of holiday dates can present challenges for businesses, planners, and individuals alike. Ultimately, the dynamic nature of our calendar system adds an element of unpredictability to our celebrations, encouraging us to adapt and find new ways to honor and enjoy these special days.

**In this analysis, we uncover which years are the best when it comes to free days within the weekdays.** We are interested only in holidays, which are fixed to a certain date. Easter, for instance, is a movable feast celebrated in Czech republic on the first Monday following the first full moon after the vernal equinox. This implies that the date of Easter changes annually but consistently occurs on a Monday.

In [2]:
import numpy as np
import pandas as pd
import calendar
from datetime import datetime, timedelta
import csv

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [10]:
# Insert holiday list based on your country

# Czech republic
holiday_list = ['01-01', '01-05', '08-05', '05-07', '06-07', '28-09', '28-10', '17-11', '25-12', '26-12']

In [4]:
def leap_year(year):
    """Calculates the leap year"""
    return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)


def days_in_year_df(year):
    """Stores the corresponding weekday of each day in a year into a dataframe"""

    #Initializing empty dictionary
    days_to_week = {'date':[], 'year':[], 'day_month':[], 'weekday':[]}

    for month in range(1, 13):
        for day in range(1, calendar.monthrange(year, month)[1] + 1):
            # Get the weekday (0 = Monday, 1 = Tuesday, ..., 6 = Sunday)
            weekday = calendar.weekday(year, month, day)
            
            # Convert weekday to the actual day name
            day_name = calendar.day_name[weekday]

            # Append the date and weekday to the dictionary
            days_to_week['date'].append(f"{day:02d}-{month:02d}-{year}")
            days_to_week['year'].append(f"{year}")
            days_to_week['day_month'].append(f"{day:02d}-{month:02d}")
            days_to_week['weekday'].append(day_name)
    
    #Create a DataFrame from the dictionary
    days_to_week_df = pd.DataFrame(days_to_week)

    # Leap year calculation
    days_to_week_df['year'] = days_to_week_df['year'].astype(str).astype(int)
    days_to_week_df['leap_year'] = days_to_week_df['year'].apply(leap_year)

    return days_to_week_df


def holidays_in_a_year(df, holiday_list):
    """Assigns a holiday dummy to all days within a year based on the provided list"""
    
    # Specifying the weekdays
    non_weekend = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    
    # Creates a holiday column with 1 or 0
    df['is_holiday'] = df['day_month'].apply(lambda x: 1 if x in holiday_list else 0)

    # Creating a column, which specifies whether a certain holiday falls on a weekday
    df['is_weekday_holiday'] = df.apply(lambda row: 1 if (row['weekday'] in non_weekend 
                                                          and row['is_holiday'] == 1) else 0, axis = 1)

In [5]:
# Generating a years list
years_list = []
for i in range(2022, 2050):    
    years_list.append(i)

# Loop through multiple years and create an aggregated dataframe
more_years = pd.DataFrame()

for years in years_list:
    more_years = pd.concat([more_years, days_in_year_df(years)], ignore_index=True)

# Assigning the is_holiday and is_weekday_holiday columns
holidays_in_a_year(more_years, holiday_list)

In [7]:
years_count = len(years_list)
year_min = min(years_list)
year_max = max(years_list)

holiday_ratio = (round(
                    len(more_years[more_years['is_weekday_holiday'] == 1])/len(more_years[more_years['is_holiday'] == 1]),
                       4) * 100)

print("From all of the holidays over last {} years ({} - {}) there was {}% of holidays on a weekday."
      .format(years_count, year_min, year_max, holiday_ratio))

From all of the holidays over last 28 years (2022 - 2049) there was 71.43% of holidays on a weekday.


In [8]:
more_years_grouped = more_years.groupby('year').aggregate({'is_weekday_holiday':'sum',
                                                           'leap_year':'max'})
more_years_grouped = pd.DataFrame(more_years_grouped.reset_index())

In [11]:
# Plotting the specified years interval
fig = px.bar(more_years_grouped,
            x="year",
            y='is_weekday_holiday',
            color='leap_year',
            labels={
                "year":"Year",
                "is_weekday_holiday":"Weekday holidays",
                "leap_year":"Leap year"
            },
            title='Weekday holidays by Year',
            category_orders={'Year': 'linear'})
#fig.update_layout(xaxis=dict(type='Year'))
fig.show()